In [ ]:
import requests
import os
import json

def download_image(url, path):
    """Downloads an image from the given URL and saves it to the given path."""
    response = requests.get(url)
    if response.status_code == 200:
        with open(path, "wb") as f:
            f.write(response.content)

def main():
    """Downloads images from Google Images."""
    search_text = input("Enter a search term: ")
    num_images = int(input("Enter the number of images to download: "))
    size = input("Enter the size of the images to download (small, medium, or large): ")

    # Get the image URLs from Google Images.
    url = "https://www.google.com/search?q=" + search_text + "&tbm=isch"
    response = requests.get(url)
    if response.status_code == 200:
        data = json.loads(response.content)
        image_urls = [image["ou"] for image in data["items"]]

    # Download the images.
    download_path = os.path.join("dataset", search_text, size)
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    for image_url in image_urls:
        download_image(image_url, os.path.join(download_path, str(num_images) + ".jpg"))
        num_images -= 1
        if num_images == 0:
            break

if __name__ == "__main__":
    main()
    

In [15]:
import sys
searchtext = sys.argv[1]

In [16]:
print(searchtext)

-f


In [17]:
searchtext = sys.argv[2]

In [21]:
print(int(searchtext))

ValueError: invalid literal for int() with base 10: '/Users/kumarmayank/Library/Jupyter/runtime/kernel-8ae034e3-fb83-4f3b-9d63-584dd6463cb0.json'

In [25]:
string = "/Users/kumarmayank/Library/Jupyter/runtime/kernel-8ae034e3-fb83-4f3b-9d63-584dd6463cb0.json"

In [26]:
new_string = string.replace('/','').replace('.', '')

In [27]:
print(int(new_string))

ValueError: invalid literal for int() with base 10: 'UserskumarmayankLibraryJupyterruntimekernel-8ae034e3-fb83-4f3b-9d63-584dd6463cb0json'

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import json
from urllib.request import *
import http.cookiejar
import sys
import time

# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()
download_path = "dataset/"

def main():
    searchtext = sys.argv[1]
    num_requested = 1000
    number_of_scrolls = num_requested / 400 + 1 
    # number_of_scrolls * 400 images will be opened in the browser

    if not os.path.exists(download_path + searchtext.replace(" ", "_")):
        os.makedirs(download_path + searchtext.replace(" ", "_"))

    url = "https://www.google.co.in/search?q="+searchtext+"&source=lnms&tbm=isch"
    driver = webdriver.Firefox()
    # from selenium import webdriver
    driver.binary = "/usr/bin/firefox"
    driver = webdriver.Firefox(firefox_options=firefox_options)
    driver.get(url)

    headers = {}
    headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
    extensions = {"jpg", "jpeg", "png", "gif"}
    img_count = 0
    downloaded_img_count = 0
    
    for _ in range(int(number_of_scrolls)):
        for __ in range(10):
            # multiple scrolls needed to show all 400 images
            driver.execute_script("window.scrollBy(0, 1000000)")
            time.sleep(0.2)
        # to load next 400 images
        time.sleep(0.5)
        try:
            driver.find_element_by_xpath("//input[@value='Show more results']").click()
        except Exception as e:
            print ("Less images found: {}".format(e))
            break

    # imges = driver.find_elements_by_xpath('//div[@class="rg_meta"]') # not working anymore
    imges = driver.find_elements_by_xpath('//div[contains(@class,"rg_meta")]')
    print ("Total images: {}\n".format(len(imges)))
    for img in imges:
        img_count += 1
        img_url = json.loads(img.get_attribute('innerHTML'))["ou"]
        img_type = json.loads(img.get_attribute('innerHTML'))["ity"]
        print ("Downloading image {}:{}".format(img_count,img_url))
        try:
            if img_type not in extensions:
                img_type = "jpg"
            req = Request(img_url, headers=headers)
            raw_img = urlopen(req).read()
            f = open(download_path+searchtext.replace(" ", "_")+"/"+str(downloaded_img_count)+"."+img_type, "wb")
            f.write(raw_img)
            f.close
            downloaded_img_count += 1
        except Exception as e:
            print("Download failed: {}".format(e))
        finally:
            if downloaded_img_count >= num_requested:
                break

    print("Total downloaded: {}/{}".format(downloaded_img_count,img_count))
    driver.quit()

if __name__ == "__main__":
    main()

SessionNotCreatedException: Message: Expected browser binary location, but unable to find binary in default location, no 'moz:firefoxOptions.binary' capability provided, and no binary flag set on the command line
